<h1>Currently the most accurate model with a score of 0.79378</h1>
It doesn't consider the cabins

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split as TSSplit
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [2]:
train_data = pd.read_csv("./train.csv")
X = train_data.iloc[:,0:13]
y = train_data.iloc[:,-1]
X.drop(['PassengerId','Name'],axis = 'columns',inplace = True)
X.columns

Index(['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP',
       'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'],
      dtype='object')

In [3]:
X.dtypes

HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
dtype: object

In [4]:
X.drop('Cabin',axis = 'columns',inplace = True)

In [6]:
one_hot = OneHotEncoder(sparse_output = False)
encoded_array = one_hot.fit_transform(X[["HomePlanet","Destination"]])
encoded_labels = one_hot.categories_
encoded_labels = np.array(encoded_labels).ravel()
encoded_features = pd.DataFrame(encoded_array,columns = encoded_labels)
t = encoded_features.columns
encoded_features.drop(t[3],axis = 1,inplace = True)
encoded_features.head()

,Earth,Europa,Mars,55 Cancri e,PSO J318.5-22,TRAPPIST-1e
0,0.0,1.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0,0.0,1.0


In [8]:
X.reset_index(drop=True, inplace=True)
encoded_features.reset_index(drop=True, inplace=True)
X = pd.concat([encoded_features,X],axis = 1)
X.drop(['HomePlanet','Destination'],axis = 1,inplace = True)
X.head()

,Earth,Europa,Mars,55 Cancri e,PSO J318.5-22,TRAPPIST-1e,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,0.0,1.0,0.0,0.0,0.0,1.0,False,39.0,False,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,False,24.0,False,109.0,9.0,25.0,549.0,44.0
2,0.0,1.0,0.0,0.0,0.0,1.0,False,58.0,True,43.0,3576.0,0.0,6715.0,49.0
3,0.0,1.0,0.0,0.0,0.0,1.0,False,33.0,False,0.0,1283.0,371.0,3329.0,193.0
4,1.0,0.0,0.0,0.0,0.0,1.0,False,16.0,False,303.0,70.0,151.0,565.0,2.0


In [13]:
X.CryoSleep = X.CryoSleep.astype('bool')
X.VIP = X.VIP.astype('bool')
X.dtypes

Earth            float64
Europa           float64
Mars             float64
55 Cancri e      float64
PSO J318.5-22    float64
TRAPPIST-1e      float64
CryoSleep           bool
Age              float64
VIP                 bool
RoomService      float64
FoodCourt        float64
ShoppingMall     float64
Spa              float64
VRDeck           float64
dtype: object

In [15]:
from xgboost import XGBClassifier
from datetime import datetime
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [49]:
xgb = XGBClassifier(learning_rate=0.02, objective='binary:logistic')
params = {
    'n_estimators' : [100,200,300,400,500,600,700,800,900,1000],
    'max_depth' : [3,4,5,6,7],
    'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
    'min_child_weight': [1, 5, 10]
}


In [52]:
# from sklearn.model_selection import GridSearchCV
# start_time = timer(None)
# grid = GridSearchCV(estimator=xgb, param_grid=params, scoring='roc_auc', n_jobs=-1, cv=3, verbose=3 )
# grid.fit(X,y)
# timer(start_time)
# print('\n All results:')
# print(grid.cv_results_)
# print('\n Best estimator:')
# print(grid.best_estimator_)
# print('\n Best score:')
# print(grid.best_score_ * 2 - 1)
# print('\n Best parameters:')
# print(grid.best_params_)
# results = pd.DataFrame(grid.cv_results_)

Best score:
0.7553022192902281

 Best parameters:
{'max_depth': 4, 'n_estimators': 500}

The above is without using the Cabins data. and min_child weight

In [22]:
X_test = pd.read_csv("./test.csv")
X_test.drop(['PassengerId','Name','Cabin'],axis = 'columns',inplace = True)
X_test.columns

Index(['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'RoomService',
       'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'],
      dtype='object')

In [24]:
encoded_array = one_hot.transform(X_test[["HomePlanet","Destination"]])
encoded_labels = one_hot.categories_
encoded_labels = np.array(encoded_labels).ravel()
encoded_features = pd.DataFrame(encoded_array,columns = encoded_labels)
t = encoded_features.columns
encoded_features.drop(t[3],axis = 1,inplace = True)
encoded_features.head()

,Earth,Europa,Mars,55 Cancri e,PSO J318.5-22,TRAPPIST-1e
0,1.0,0.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,1.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0,0.0,1.0


In [25]:
X_test.reset_index(drop=True, inplace=True)
encoded_features.reset_index(drop=True, inplace=True)
X_test = pd.concat([encoded_features,X_test],axis = 1)
X_test.drop(['HomePlanet','Destination'],axis = 1,inplace = True)
X_test.head()

,Earth,Europa,Mars,55 Cancri e,PSO J318.5-22,TRAPPIST-1e,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,1.0,0.0,0.0,0.0,0.0,1.0,True,27.0,False,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,False,19.0,False,0.0,9.0,0.0,2823.0,0.0
2,0.0,1.0,0.0,1.0,0.0,0.0,True,31.0,False,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,1.0,False,38.0,False,0.0,6652.0,0.0,181.0,585.0
4,1.0,0.0,0.0,0.0,0.0,1.0,False,20.0,False,10.0,0.0,635.0,0.0,0.0


In [27]:
X_test.CryoSleep = X_test.CryoSleep.astype('bool')
X_test.VIP = X_test.VIP.astype('bool')
X_test.dtypes

Earth            float64
Europa           float64
Mars             float64
55 Cancri e      float64
PSO J318.5-22    float64
TRAPPIST-1e      float64
CryoSleep           bool
Age              float64
VIP                 bool
RoomService      float64
FoodCourt        float64
ShoppingMall     float64
Spa              float64
VRDeck           float64
dtype: object

In [28]:
test_data = pd.read_csv("./test.csv")
test_data.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name'],
      dtype='object')

In [39]:
y_test = grid.best_estimator_.predict_proba(X_test)
threshold = 0.5
y_test_binary = (y_test[:, 1] >= threshold).astype(int)
results_df = pd.DataFrame(data={'PassengerId':test_data['PassengerId'], 'Transported':y_test_binary})
results_df.head()
# results_df.to_csv('submission-grid-search-xgb-porto-01.csv', index=False)

,PassengerId,Transported
0,0013_01,1
1,0018_01,0
2,0019_01,1
3,0021_01,1
4,0023_01,1


In [42]:
results_df.Transported = results_df.Transported.astype('bool')
results_df.dtypes

PassengerId    object
Transported      bool
dtype: object

In [44]:
results_df.head()
results_df.to_csv('./submission.csv', index=False)

In [45]:
test_data.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name'],
      dtype='object')

In [51]:
#Now adding Cabins to the mix
X.to_csv('Train_X.csv', index=False)
X_test.to_csv("Test_X.csv",index = False)